Search Data Processing
=========================



In [1]:
import numpy as np
import pandas as pd
import re
import string
import pyspark

In [2]:
search_data = pd.read_csv("./data/sample_openData_searchTerms_clean.csv")

In [3]:
search_data.shape

(5000, 8)

In [4]:
search_data.tail()

,Search Term,Exit Page,Total Unique Searches,Results Pageviews / Search,% Search Exits,% Search Refinements,Time after Search,Average Search Depth
4995,address range,/Geographic-Locations-and-Boundaries/Streets-o...,2,1.5,0.00%,0.00%,0:02:09,8.0
4996,address shapefile,/browse/embed?category=&limit=20&limitTo=&q=ad...,2,1.5,0.00%,66.67%,0:00:18,3.0
4997,address to geographic location,/browse/embed?category=&limit=20&limitTo=&q=ad...,2,1.0,0.00%,0.00%,0:00:03,1.5
4998,address with resident names,/browse/embed?category=&limit=20&limitTo=&q=ad...,2,1.0,0.00%,0.00%,0:00:05,1.5
4999,addresse,/browse/embed?category=&limit=20&limitTo=&q=po...,2,1.0,0.00%,100.00%,0:00:03,1.5


In [5]:
#search_data[search_data['Total Unique Searches']<5]

In [6]:
search_terms = list(set(search_data['Search Term']))

In [7]:
len(search_terms)

2451

In [8]:
#print search_terms

In [9]:
search_data.ix[search_data["Search Term"] == "194415"]

,Search Term,Exit Page,Total Unique Searches,Results Pageviews / Search,% Search Exits,% Search Refinements,Time after Search,Average Search Depth
3519,194415,/Economy-and-Community/Registered-Business-Loc...,2,1,0.00%,0.00%,0:00:06,2.5


In [10]:
search_terms_data = search_data[["Search Term"]]
search_terms_data = search_terms_data.rename(columns={"Search Term": "search_term"})

In [11]:
search_terms_data.head()

,search_term
0,business
1,crime
2,crime
3,311
4,streets


In [12]:
search_terms_data['processed_data'] = search_terms_data.search_term\
                                        .apply(lambda text: text.decode('ascii' ,"ignore" ).decode('utf-8','ignore'))\
                                        .apply(lambda text: text.lower())

In [13]:
search_terms_list =  list(set(search_terms_data.processed_data))
#print search_terms_list

#### We need to generally tag searches in categories of quality

Many search terms indicate that users are not informed on what the purpose of the website is or how to properly use search

Search Tags
* Good Quality Search
    * complete words or phrases
    * minor typos
* Bad Quality
    * Addresses
    * Dates
    * Zipcodes or just a string of numbers
    * General nonsenses eg. ('></script><script>alert(1)</script>', '///', '16exc-3031') 

In [14]:
dates_r = re.compile('[0-9]/[0-9]/[0-9]')
numbers_r = re.compile('^[0-9][0-9]*[0-9]$')
html_r = re.compile('^<.*>$')


In [15]:
filter(dates_r.match, search_terms_list)

[u'9/1/01', u'3/1/01']

In [16]:
print filter(html_r.match, search_terms_list)

[u'</script><script>alert(1)</script>']


In [17]:
# removing punctuation

def removePunctuation(text):

    for c in string.punctuation:
        text = text.replace(c,"").strip().lower()
    return text

In [18]:
# iterative process

def text_processing(search):
    
    return [removePunctuation(i) for i in search]
        
        

In [19]:
search = text_processing(search_terms_list)
#search

In [20]:
regex = "\d{1,4}.?\d{0,4}\s[a-zA-Z|\d+]{2,30}\s[a-zA-Z]|\s[a-zA-Z]*"

f = [re.findall(regex, i) for i in search
     if re.findall(regex, i)!= [] 
     if re.findall(regex, i)[0][:3] != '311'
     ]

# http://regexlib.com/REDetails.aspx?regexp_id=430

In [21]:
year = [str(j) for j in range(2000,2017)]

addresses = [i for i in f if i[0][:4] not in year ]
#addresses

In [22]:
# applying existing code to the full data

query = pd.read_csv("./data/all_queries.csv")

In [23]:
query.head()

,ga.searchKeyword,ga.searchStartPage,ga.searchAfterDestinationPage,ga.searchUniques,ga.avgSearchResultViews,ga.avgSearchDepth,ga.percentSearchRefinements,ga.searchDuration,ga.searchExitRate
0,crime,'/,'/data?search=crime,451,1.066519,0.00,0.000000,62,0
1,business,'/,'/data?search=business,319,1.106583,0.00,0.000000,5,0
2,311,'/,'/data?search=311,221,1.135747,0.00,0.000000,7,0
3,crime,'/,'/browse/embed?Department-Metrics_Publishing-D...,200,1.070000,4.54,12.149533,48646,0
4,streets,'/,'/data?search=streets,169,1.047337,0.00,0.000000,3,0


In [24]:
len(query)

116339

In [25]:
query.shape


(116339, 9)

In [26]:
# seeing the data

query_list =  list(query['ga.searchKeyword'].values)
query_list = [str(word).decode('ascii' ,"ignore" ).decode('utf-8','ignore') for word in query_list]

In [27]:
set_query = len(set(query_list))
set_query

20600

In [28]:
#adresses
regex = "\d{1,4}.?\d{0,4}\s[a-zA-Z|\d+|\W+]{2,30}\s[a-zA-Z]{2,15}"

full = [re.findall(regex, i) for i in query_list
     if re.findall(regex, i)!= [] 
     if re.findall(regex, i)[0][:3] != '311'
     ]


In [29]:
def search_term_type(text):
    regex = "\d{1,4}.?\d{0,4}\s[a-zA-Z|\d+|\W+]{2,30}\s[a-zA-Z]{2,15}"
    links = "((https?|http):((//)|(\\\\))+[\w\d:#@%/;$()~_?\+-=\\\.&]*)|www.*"
    #link = '[\w\d:#@%/;$()~_?\+-=\\\.&]*'
    years = [str(i) for i in range(2000,2017)]
    if len(re.findall(regex, text))> 0 and re.findall(regex, text)[0][:4] not in years and re.findall(regex, text)[0][:3] != '311':
        return "Address"
    if len(re.findall(links, text))> 0:
        return "Link"
    if len(re.findall(regex, text))> 0 and re.findall(regex, text)[0][:4] in years:
        return "Year"
    
    return "Valid Search Term"

In [30]:
years = [str(i) for i in range(2000,2017)]

In [31]:
string = 'data for crime'
search_term_type(string)

'Valid Search Term'

In [34]:
query['Search Type'] = ['blank'] * len(query)

In [35]:
keywords = list(query['ga.searchKeyword'].values)

In [36]:
types = [search_term_type(str(word)) for word in keywords]

In [37]:
query['Search Type'] = types

In [ ]:
query.to_csv('all_queries_w')

In [38]:
Address = query[query['Search Type'] == 'Address']
Address 

,ga.searchKeyword,ga.searchStartPage,ga.searchAfterDestinationPage,ga.searchUniques,ga.avgSearchResultViews,ga.avgSearchDepth,ga.percentSearchRefinements,ga.searchDuration,ga.searchExitRate,Search Type
1552,460 forms Schedule A,'/,'/browse/embed?category=&limit=20&limitTo=&q=4...,3,1,4.333333,0,1982,0,Address
2364,"1010 fitzgerald ave, san francisco, ca",'/,"'/data?search=1010+fitzgerald+ave,+san+francis...",2,1,0.000000,0,0,0,Address
2365,1600 California street,(entrance),'/data?search=1600+California+street,2,1,0.000000,0,0,0,Address
2366,1996-2013 city survey,'/,'/browse/embed?category=&limit=20&limitTo=&q=1...,2,1,3.000000,0,77,0,Address
2367,2 hour parking,(entrance),'/browse/embed?category=&limit=20&limitTo=&q=2...,2,1,2.500000,50,65,0,Address
2368,2 hour parking,(entrance),'/data?search=2+hour+parking,2,1,0.000000,0,0,0,Address
2379,2598 Mission Street,'/,'/data?search=2598+Mission+Street,2,1,0.000000,0,0,0,Address
2403,405 howard st.,(entrance),'/browse/embed?category=Energy+and+Environment...,2,1,2.000000,0,19,0,Address
2417,5 thomas mellon circle,'/,'/data?search=5+thomas+mellon+circle,2,1,0.000000,0,0,0,Address
4879,"""120 lake street""",'/,"'/data?search=""120+lake+street""",1,1,0.000000,0,0,0,Address


In [39]:
text = '2010 us census data'
regex = "\d{1,4}.?\d{0,4}\s[a-zA-Z|\d+|\W+]{2,30}\s[a-zA-Z]{2,15}"
r = re.findall(regex, text)[0][:4]
r in range(2000,2017)

False

In [40]:
# flatten full list
full = [j for i in full for j in i]
full

[u'2010 census tracts',
 u'460 forms Schedule',
 u'1010 fitzgerald ave, san francisco, ca',
 u'1600 California street',
 u'1996-2013 city survey',
 u'2 hour parking',
 u'2 hour parking',
 u'2011 herrera campaign contributions',
 u'2012 SFO Customer Survey',
 u'2013 housing inventory',
 u'2598 Mission Street',
 u'405 howard st',
 u'5 thomas mellon circle',
 u'2000 Blocks (no water',
 u'2000 Blocks (no water',
 u'2000 Tracts (no water',
 u'2000 Tracts (no water',
 u'2000 Blocks (no water',
 u'2000 blocks (no water',
 u'2000 tracts (no water',
 u'2000 tracts (no water',
 u'120 lake street',
 u'0 Beatrice Rd, Brisbane, CA',
 u'19/2015 dashiell hammet',
 u'1 Embarcadero San Francisco',
 u'1 Hawkins Ln SF CA',
 u'1 Longview Court',
 u'1 Polk st',
 u'1 Tuscany Alley',
 u'1 hour parking',
 u'1 post st',
 u'1 scott street',
 u'1 south van ness ave',
 u'1 south vanness',
 u'1. BRIAN DUSSEAULT',
 u'10 gb dataset',
 u'10 glendale street',
 u'100 church street',
 u'100 delano ave',
 u'100 year floo

In [41]:
year_full = [str(j) for j in range(2000,2017)]
# taking out the years

addresses_full = [i for i in full if i[:4] not in year ]
addresses_full = list(set(addresses_full))

In [42]:
year_listing = [i.lower() for i in full if i[:4] in year ]
year_listing = list(set(year_listing))
year_listing

[u'2000 block group',
 u'2007 housing inventory',
 u'2010 census block for san francisco',
 u'2000 blocks (no water',
 u'2010 census tracts no water',
 u'2010: census blocks for san',
 u'2016 assessor map',
 u'2010 census population',
 u'2015 crime statistics',
 u'2014 energy benchmark 285 geary',
 u'2009 sfo survey',
 u'2015 enacted budget',
 u'2010 tract population',
 u'2015 housing inventory',
 u'2013 city survey report',
 u'2012 campaign finance',
 u'2005 pipeline report',
 u'2000 tracts no water',
 u'2014 city serve report',
 u'2016 budget publication',
 u'2000 census tract',
 u'2011 mayor campaign contributions',
 u'2000 block goup',
 u'2012 sfo customer survey',
 u'2015 richmond district',
 u'2010 demographic profile data, sf county',
 u'2015 in san francisco',
 u'2015 form 460; schedule',
 u'2014 map:crime incidents',
 u'2011 herrera campaign contributions',
 u'2015 residential development pipeline',
 u'2004 bicycle network',
 u'2010 vehicle ownership',
 u'2008 crime data',
 u'

In [43]:
#clean_list = sc.parallelize(query_list).filter(lambda word: word.lower() not in addresses_full).collect()

In [44]:
clean_list = [i for i in query_list if i.lower() not in addresses_full]
#clean_list

In [88]:
links = "((https?|http):((//)|(\\\\))+[\w\d:#@%/;$()~_?\+-=\\\.&]*)"

https = [re.findall(links, i) for i in clean_list
     if re.findall(links, i)!= [] 
     if re.findall(links, i)!= "//" 
     ]
# http://stackoverflow.com/questions/6718633/python-regular-expression-again-match-url

In [89]:
# finding all the unique links that people have put into the search query

# https = (sc.parallelize(https).flatMap(lambda word: word)
#         .flatMap(lambda word: word)
#         .filter(lambda word: word != '//')
#         .filter(lambda word: word != 'https')
#         .filter(lambda word: word != '''''')
#         .filter(lambda word: word != 'http')
#         .collect())

In [92]:
https = [i[0] for i in https]

In [93]:
https

[u'https://data.sfgov.org/Public-Health/Excessive-Rent-Burden-San-Francisco-CA/9wty-qwgq',
 u'https://data.sfgov.org/Public-Health/Restaurant-Scores/stya-26eb',
 u'http://googlewebmastercentral.blogspot.com/2014/11/helping-users-find-mobile-friendly-pages.html',
 u'http://www.bsis.ca.gov/forms_pubs/fire_fact.shtml',
 u'http://www.datasf.org/story.php?title=street-sweeper-schedule-and-route-',
 u'https://10.183.241.201/rgcertprintv2default.aspx',
 u'https://data.sfgov.org/Economy-and-Community/Off-Sale-Alcohol-Outlets-San-Francisco-CA/fIy-9zhp',
 u'https://data.sfgov.org/Economy-and-Community/Registered-Business-Locations-San-Francisco/g8m3-pdis',
 u'https://data.sfgov.org/Economy-and-Community/Registered-Business-Locations-San-Francisco/g8m3-pdis?',
 u'https://data.sfgov.org/d/ejmn-jyk6',
 u'https://data.sfgov.org/data?category=',
 u'https://data.sfgov.org/data?category=Transportation',
 u'https://extxfer.sfdph.org/food/',
 u'https://www.fiverr.com/kawsarhossain',
 u'https://www.kaggle

In [40]:
# clean_list = (sc.parallelize(query_list).map(lambda word: word.lower()).filter(lambda word: word not in https)
#               .filter(lambda word: word not in addresses_full)
#               .collect())

In [95]:
clean_list = [word for word in query_list if word not in https if word not in addresses_full]

In [96]:
from collections import Counter
word_count = Counter(clean_list)
word_count_sorted = sorted(word_count.items(),key = lambda x: x[1], reverse=True)

In [97]:
word_count_sorted = [i[0] for i in word_count_sorted if i[1]>2]
word_count_sorted

[u'crime',
 u'streets',
 u'shapefile',
 u'parking',
 u'census',
 u'street',
 u'bike',
 u'business',
 u'building',
 u'311',
 u'population',
 u'parks',
 u'neighborhood',
 u'water',
 u'bicycle',
 u'traffic',
 u'income',
 u'parcel',
 u'fire',
 u'housing',
 u'school',
 u'rent',
 u'muni',
 u'elevation',
 u'buildings',
 u'homeless',
 u'parcels',
 u'land use',
 u'neighborhoods',
 u'bus',
 u'zoning',
 u'restaurant',
 u'boundary',
 u'food',
 u'bart',
 u'roads',
 u'park',
 u'address',
 u'pipeline',
 u'police',
 u'schools',
 u'san francisco',
 u'budget',
 u'property',
 u'transit',
 u'assessor',
 u'city',
 u'gis',
 u'trees',
 u'restaurants',
 u'bridge',
 u'construction',
 u'shapefiles',
 u'pedestrian',
 u'block',
 u'businesses',
 u'taxi',
 u'transportation',
 u'building footprints',
 u'health',
 u'business license',
 u'tree',
 u'building footprint',
 u'district',
 u'graffiti',
 u'education',
 u'permit',
 u'census tract',
 u'car',
 u'building permits',
 u'blocks',
 u'sfmta',
 u'road',
 u'airport',
 

In [98]:
string_word = removePunctuation(re.sub("\u" , '', str(word_count_sorted)))
string_word1 = removePunctuation(re.sub("\u" , '', str(query_list)))

In [99]:
from polyglot.text import Text

NER = Text(string_word1)

NER = NER.entities

for entity in NER:
    if entity.tag == "I-PER":
        print entity
    


[u'bart', u'bildings']
[u'bart']
[u'san', u'francisco']
[u'francisco']
[u'francisco']
[u'polltion']
[u'bart', u'bart', u'bart']
[u'bsiness']
[u'conties']
[u'crosswalk', u'crb']
[u'lidar']
[u'francisco']
[u'bart', u'bart']
[u'bart', u'bart', u'basemap']
[u'bsiness']
[u'conty', u'bondary', u'cort']
[u'lyft', u'maher']
[u'francisco']
[u'san']
[u'otline', u'san']
[u'sanfrancisco']
[u'ber', u'ber', u'nemployment']
[u'xls']
[u'fitzgerald']
[u'san']
[u'herrera']
[u'howard']
[u'thomas', u'mellon']
[u'ada']
[u'apns']
[u'bart', u'bart', u'bart', u'bart']
[u'beaty']
[u'francisco']
[u'gdb', u'san']
[u'sanfrancisco']
[u'taz', u'taz', u'taz']
[u'adrian', u'santos']
[u'ariel']
[u'bart', u'bart', u'bart', u'bart', u'bart', u'bart']
[u'bart']
[u'bsines']
[u'contor']
[u'crbs']
[u'david', u'owen']
[u'ellis', u'ellis']
[u'formla']
[u'contribtions']
[u'lyft', u'lyft', u'maher', u'maher', u'maher']
[u'msems']
[u'nancy', u'jeng']
[u'noe']
[u'francisco']
[u'francisco']
[u'francisco']
[u'francisco']
[u'san']
[

# important people:
- dennis herrera , city attorney
- aaron peskin , sf supervisor
- jane kim , SF District 6 Supervisor
- hilary ronen , SF District 9 supervisor
- raymond chow , san francisco gangster/felon (high possibility)
- Kenneth Malvar , City Sightseeing operator (high possibility)
http://www.sfgate.com/bayarea/article/Driver-blamed-for-tour-bus-crash-in-SF-s-Union-6994435.php


- Blake Rawdin , private MD(possibly)
- Antonin Scalia, Supreme Court Justice (possibly)
- charle, walton Co-Founder and Partner, Kindred Partners, LLC (possibly)
- Henry albert , private eye doctor(possibly)
- melanie lok , owner of mlok consulting private consulting (possibly)
- bateman, bateman group(possibly)
- David Owen , SF Attorney(possibly)
- Jonah Yee, Private Eye Doctor(possibly)
- Molly Seager, Private Therapist(possibly)
- anson mooney(arrested)
https://jailalert.com/arrest-records/anson-mooney-310655.html
- alain gervais, private owner salon

In [45]:
#not_caught = [ '3180 18th street' , '100 church', '17th street' , '800 university avenue, palo alto, california', '17 san andreas way, san francisco' , '2631 23rd']

In [46]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF

In [47]:
vec=CountVectorizer(stop_words='english',analyzer='word')
X_train_counts = vec.fit_transform(word_count_sorted)
vocab = vec.get_feature_names()
nmf = NMF(n_components = 10, random_state = 1)
nmf.fit(X_train_counts)

NMF(alpha=0.0, beta=1, eta=0.1, init=None, l1_ratio=0.0, max_iter=200,
  n_components=10, nls_max_iter=2000, random_state=1, shuffle=False,
  solver='cd', sparseness=None, tol=0.0001, verbose=0)

In [48]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [49]:
print(print_top_words(nmf, clean_list, 50))

Topic #0:
green space high injury corridor churches odata open spaces employment movie pedestrian inventory blocks city attorney trees data inventory curbs census tract from 460 rape unemployment home recycling schedule e curb business licenses demographic bay area fire incidents school crime assessor inventory hiv crime homeless collisions crime rate trees building footprint shapefiles pipeline homicide construction murder bridges san francisco buildings buildings land use sanfrancisco.gdb census tracts wind
Topic #1:
muni city lots bus stops bridges inventory business account number mental health crime trees parcels curb city boundary curb building permit neighborhood 94109 intersection active businesses odata pipeline bart human waste census neighborhoods topography bart motionloft murder resiliency income park muni sfo elections sf blocks parking crime classification budget median income open spaces business registration certificate 460 schedule a property assessment roll address c

In [50]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(word_count_sorted)
indices = np.argsort(vectorizer.idf_)
features = vectorizer.get_feature_names()
top_n = 100
top_features = [features[i] for i in indices[:top_n]] 

print top_features

[u'san', u'francisco', u'street', u'city', u'business', u'data', u'shapefile', u'of', u'sf', u'census', u'building', u'map', u'parking', u'and', u'bay', u'crime', u'housing', u'water', u'bike', u'district', u'area', u'public', u'property', u'traffic', u'block', u'in', u'streets', u'food', u'income', u'population', u'by', u'car', u'code', u'campaign', u'open', u'neighborhood', u'bicycle', u'gis', u'use', u'districts', u'2015', u'community', u'tax', u'buildings', u'health', u'transit', u'school', u'park', u'fire', u'planning', u'number', u'permit', u'pipeline', u'2000', u'market', u'land', u'permits', u'2010', u'for', u'muni', u'parks', u'2014', u'bus', u'commercial', u'center', u'police', u'lines', u'parcel', u'zip', u'county', u'rent', u'311', u'sfpd', u'service', u'rate', u'facilities', u'finance', u'460', u'blocks', u'businesses', u'services', u'construction', u'zoning', u'locations', u'assessor', u'restaurant', u'schedule', u'residential', u'space', u'boundary', u'development', u'ro

In [51]:
import pickle

In [52]:
pickle.dump(word_count_sorted , open("word_count_sorted.p",'wb'))

# Recommendations:

- spell corrector for mispelled words
- recommendations for words while typing of words of correct search keywords, fuzzy matching
- education for how to properly use the open data portal

To do:
    
- create a column
- functions to tag addresses,links, bad words things i wrote regex for
- add to tag to column 